In [ ]:
!pip freeze

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from itertools import combinations

# Leer datos
## lista de reproduccion de spotify


In [132]:
data = np.load('/content/spotify.npy', allow_pickle=True)
# print(data)

## convertir npy a dataframe

In [144]:
spotify = dict()
spotify_arr = list()

k = 0
for i, item in enumerate(data.item()):
    spotify['transaction' + str(i)] = data.item()[i]
    if k <= 10:
        spotify_arr.append(data.item()[i])
    k += 1


In [101]:
df = pd.DataFrame(list(spotify.items()), columns=['transaction', 'music set'])

## 10 transacciones 

In [134]:
df.head(10)

,transaction,music set
0,transaction0,"[Lose Control (feat. Ciara & Fat Man Scoop), T..."
1,transaction1,"[Eye of the Tiger, Libera Me From Hell (Tengen..."
2,transaction2,"[Like You, GOOD (feat. ELO), Inferiority Compl..."
3,transaction3,"[Danse macabre, Piano concerto No. 2 in G Mino..."
4,transaction4,"[Tonight, Tonight, Wonderwall - Remastered, I ..."
5,transaction5,"[Teach Me How to Dougie, Party In The U.S.A., ..."
6,transaction6,"[I Put A Spell On You, Bury Us Alive, Jackolan..."
7,transaction7,"[Hard To See You Happy, One Thousand Times, So..."
8,transaction8,"[Twice, 7, Whole Wide World - Unpeeled, Goodby..."
9,transaction9,"[Highwayman, Highwayman, Drunk Like You, Fix, ..."


In [126]:
for val in spotify_arr:
    print(val)

[['Lose Control (feat. Ciara & Fat Man Scoop)', 'Toxic', 'Crazy In Love', 'Rock Your Body', "It Wasn't Me", 'Yeah!', 'My Boo', 'Buttons', 'Say My Name', 'Hey Ya! - Radio Mix / Club Mix', 'Promiscuous', 'Right Where You Want Me - Radio Edit Version', 'Beautiful Soul', "Leavin'", 'Me & U', 'Ice Box', 'Sk8er Boi', 'Run It!', 'Check On It - feat. Bun B and Slim Thug', "Jumpin', Jumpin'", 'Soak Up The Sun', 'Where Is The Love?', "Stacy's Mom", 'Just The Girl', 'Yo (Excuse Me Miss)', 'Year 3000', 'Lip Gloss', 'Everytime We Touch - Radio Edit', 'Whatcha Say', 'Miss Independent', 'Party In The U.S.A.', 'The Great Escape', 'Replay', 'Forever', 'Your Love Is My Drug', 'Closer', 'One Less Lonely Girl', 'Paper Planes', 'Mr. Brightside', 'All The Small Things', 'Beep', 'Somebody To Love', 'Dirty Little Secret', 'Baby', 'A Thousand Miles', 'Livin on Sunday', 'See You Again', 'How Do You Sleep? - Featuring Ludacris', 'This Is Me', 'My Happy Ending', 'Check Yes Juliet', 'The Great Escape'], ['Eye of t

## datos de prueba para en la implementación del algoritmo

In [129]:
data = [
        ['bread', 'milk'],
        ['bread', 'diapers', 'beer', 'eggs'],
        ['milk', 'diapers', 'beer', 'cola'],
        ['bread', 'milk', 'diapers', 'beer'],
        ['bread', 'milk', 'diapers', 'cola']]

# Implementación del algoritmo apriori

función que crea combinaciones de candidatos con $(k > 1)$

In [45]:
def create_candidate(freq_item, k):
    ans = []
    if k > 0:
        for set1, set2 in combinations(freq_item, 2):
            if len(set1 & set2) == k:
                item = set1 | set2
                if item not in ans:
                    ans.append(item)
    else:
        for set1, set2 in combinations(freq_item, 2):
            item = set1 | set2
            ans.append(item)
    return ans

función que crea la frecuencia de k-items y en base a los candidatos que obtienen de la funcion anterior que tengan el $soporte \ge minsuport$

In [4]:
def create_freq(data, ck, min_support):
    counted = {}
    for tran in data:
        for item in ck:
            if item.issubset(tran):
                if item not in counted:
                    counted[item] = 1
                else: counted[item] += 1
    
    N = len(data)
    freq_item = []
    item_suport = {}
    for item in counted:
        support = counted[item]/N
        if support >= min_support:
            freq_item.append(item)
        item_suport[item] = support
    return freq_item, item_suport

función que genera el con todos los conjuntos de items que $σ > N(minsuport)$

In [46]:
def get_frequent_itemsets(data, min_support):

    counted = {}
    for transaction in data:
        for item in transaction:
            if item not in counted:
                counted[item] = 1
            else: counted[item] += 1

    N = len(data)
    candidate = list()
    for item in counted:
        support = counted[item]/N
        if support >= min_support:
            candidate.append(frozenset([item]))

    # candidate = find_one_itemsets(data, min_support)
    
    freq_item, item_support_dict = create_freq(data, candidate, min_support)
    freq_itemsets = [freq_item]
    k = 0
    while len(freq_itemsets[k]) > 0:
        freq_item = freq_itemsets[k]
        k_candidates = create_candidate(freq_item, k)
        freq_item, item_support = create_freq(data, k_candidates, min_support)
        freq_itemsets.append(freq_item)
        item_support_dict.update(item_support)
        k += 1
    # freq_items = list(map(lambda fset: list(map(list, fset)), freq_itemsets))

    # for key, val in list(item_support_dict.items()):
    #     item_support_dict[tuple(key)] = item_support_dict.pop(key)
    
    return freq_itemsets, item_support_dict

función que filtra $confidence$ y $lift$

In [68]:
def find_confindence_lift(freq_items, item_support_dict, freq_set, subsets, min_confidence, min_lift):

    rules = []
    consequent = []
    
    for rhs in subsets:
        lhs = freq_set - rhs
        conf = item_support_dict[freq_set] / item_support_dict[lhs]
        if conf >= min_confidence:
            lift = conf / item_support_dict[rhs]
            if lift >= min_lift:
                rules_info = lhs, rhs, item_support_dict[rhs], conf, lift
                rules.append(rules_info)
                consequent.append(rhs)
            
    return rules, consequent

Función que genera reglas de asociacion en base al frecuanica de los itemsets, con parametors (data, filtros: support, confidence, lift), retorna data frame de pandas. 

In [73]:
def generate_association_rules(data, min_support=0.5, min_confidence=0.5, min_lift=1.2):
    frequent_itemsets, item_support_dict = get_frequent_itemsets(data, min_support)
    association_rules = []
    for i, freq_item in enumerate(frequent_itemsets[1:(len(frequent_itemsets) - 1)]):
        for freq_set in freq_item:
            
            subsets = [frozenset([item]) for item in freq_set]
            rules, consequent = find_confindence_lift(frequent_itemsets, item_support_dict, freq_set, subsets, min_confidence, min_lift)
            association_rules.extend(rules)
            
            if i != 0:
                k = 0
                while len(consequent) > 0 and len(consequent[0]) < len(freq_set) - 1:
                    ck = create_candidate(consequent, k = k)
                    rules, consequent = find_confindence_lift(frequent_itemsets, item_support_dict, freq_set, ck, min_confidence, min_lift)
                    association_rules.extend(rules)
                    k += 1    
    association_rules = list(map(lambda fset: list(map(lambda item: list(item) if type(item) is frozenset else item, fset)), association_rules))

    return pd.DataFrame(association_rules, columns=['itemset1', 'itemst2', 'support', 'confidence', 'lift'])

# Ejecución de la prueba de con un grupo pequeño de datos *spotify*

In [146]:
spotify_association_rules = generate_association_rules(spotify_arr, min_support=0.1, min_confidence=0.5, min_lift=0.9)
spotify_association_rules.head(25)


,itemset1,itemst2,support,confidence,lift
0,[Party In The U.S.A.],[Yeah!],0.181818,1.0,5.5
1,[Yeah!],[Party In The U.S.A.],0.181818,1.0,5.5
2,[Yeah!],[Forever],0.181818,1.0,5.5
3,[Forever],[Yeah!],0.181818,1.0,5.5
4,[Party In The U.S.A.],[Forever],0.181818,1.0,5.5
5,[Forever],[Party In The U.S.A.],0.181818,1.0,5.5
6,"[Yeah!, Party In The U.S.A.]",[Forever],0.181818,1.0,5.5
7,"[Forever, Party In The U.S.A.]",[Yeah!],0.181818,1.0,5.5
8,"[Yeah!, Forever]",[Party In The U.S.A.],0.181818,1.0,5.5
9,[Party In The U.S.A.],"[Yeah!, Forever]",0.181818,1.0,5.5


En los resultados obtenidos tenemos que si una persona que escucha las canciones { Forever, Party In The U.S.A } implica que tambine puede escuhar el tema { Yeah! } ya que tiene un $(lift > 1)$